In [85]:
import pandas as pd

#Merge 3 dataframes into 1                
df = pd.read_csv("./data/User_track_data.csv")
df1 = pd.read_csv("./data/User_track_data_2.csv")
m1 = pd.merge(df,df1,how="outer") # m1 = df+df1
df2 = pd.read_csv("./data/User_track_data_3.csv")
mega_df = pd.merge(m1,df2,how="outer") # mega_df = df2+m1

print(mega_df.tail(5))

    User_name                  User_ID   Artist_name                Track  \
295         J  jimenezzuletajhonjanner    Katy Perry     California Gurls   
296         J  jimenezzuletajhonjanner         Adele  Rolling in the Deep   
297         J  jimenezzuletajhonjanner   John Legend            All of Me   
298         J  jimenezzuletajhonjanner  Fools Garden           Lemon Tree   
299         J  jimenezzuletajhonjanner     Passenger             Let's Go   

     valence  
295    0.425  
296    0.507  
297    0.331  
298    0.643  
299    0.649  


In [86]:
class User:
    def __init__(self, user_name: str, user_id: int, user_mean: int) -> None:
        self.left = None
        self.right = None
        self.user_name = user_name
        self.height = 1
        self.user_id = user_id
        self.user_mean = user_mean
    
    def __str__(self):
        return f'[+] User name: {self.user_name}\n    User ID:{self.user_id}\n    Valence mean: {self.user_mean}'

In [87]:
class ArbolBinarioBalanceado:
    def __init__(self,nodos=None) -> None:
        '''
        Iniciamos el arbol binario balanceado con una raíz
        '''
        self.root = None
        # if nodos is not None:
        #     self.agregar_nodos(nodos)
    
    def agregar_nodo(self, nodo: User):
        '''
        Permite agregar un nodo al arbol

        Args: 
            value (int) : [Value of a node]
        '''
        # Le agrega a la raíz el siguiente nodo el cual lo agregaremos de manera recursiva en la funcion "agregar_nodo_recursivamente"
        self.root = self.agregar_nodo_recursivamente(self.root, nodo)
        # print(f'Se acaba de agregar {value}')
    
    # def agregar_nodos(self,nodos):
    #     """
    #     Permite agregar varios nodos al tiempo

    #     Args:
    #         nodos(list) : [lista de valores a insertar]
    #     """
    #     for x in nodos:
    #         self.root = self.agregar_nodo_recursivamente(self.root,x)

    def agregar_nodo_recursivamente(self, current: User, nodo: User) -> User:
        '''
        Permite agregar de manera recursiva un nodo al arbol

        Args:
            current (int) : [Current node]
            value   (int) : [Vale of the node we want to add]
        
        Returns:
            Node : Node we're going to add
        '''

        # Preguntamos que si el nodo actual es distinto de None, que me retorne el nodo con el valor que
        # queremos agregar (recordar que retorna un nodo porque lo queremos usar en el método de arriba llamado
        # agregar_nodo)
        if not current:
            return nodo

        # Si el valor es menor al valor del nodo actual, ira a la izquierda    
        elif nodo.user_mean < current.user_mean:
            current.left = self.agregar_nodo_recursivamente(current.left, nodo)
        # Misma lógica para el lado derecho
        else:
            current.right = self.agregar_nodo_recursivamente(current.right, nodo)
        
        # Vamos a asignarle la altura al nodo 
        current.height = 1 + max(self.get_hight(current.left), self.get_hight(current.right))
        
        # Falculamos el factor de equilibrio de los nodos a los caules estamos recorriendo para verificar
        # de inmediato si se deben realizar rotaciones o no
        factor_equilibrio = self.balanced_factor(current)
        
        # Aqui entra la lógica para que se autobalancee 

        # Aqui considero si el factor de equilibrio es 2 o más y que el valor del nodo que este agregando
        # sea menor que la hoja (creo que todos saben que es una hoja), si va a la izquierda es que se va a desbalancear
        # hacia la izq así que hacemos una rotación derecha

        # Definición de Hoja: Nodo sin hijos

        if factor_equilibrio > 1 and nodo.user_mean < current.left.user_mean:
            return self.rotacion_derecha(current)
        
        # Aquí lo mismo pero cuando se va a desbalancear hacia la derecha
        if factor_equilibrio < -1 and nodo.user_mean > current.right.user_mean:
            return self.rotacion_izquierda(current)
        
        # Se realiza una rotación doble derecha (condiciones que me dijo chatgpt)
        if factor_equilibrio > 1 and nodo.user_mean > current.left.user_mean:
            current.left = self.rotacion_izquierda(current.left)
            return self.rotacion_derecha(current)
        
        # Se realiza una rotación doble izquierda (condiciones que me dijo chatgpt)
        if factor_equilibrio < -1 and nodo.user_mean < current.right.user_mean:
            current.right = self.rotacion_derecha(current.right)
            return self.rotacion_izquierda(current)
        
        return current
    
    def eliminar_nodo(self,root,value):
        '''
        Permite agregar de manera recursiva un nodo al arbol

        Args:
            root (int) : [Current node]
            value   (int) : [Vale of the node we want to delete]
        
        '''
        # Borrado de un BST tradicional
        if root is None: 
            return root
        if value < root.value: #Avanzamos por el arbol
            root.left = self.eliminar_nodo(root.left,value)
        elif value>root.value:
            root.right = self.eliminar_nodo(root.right,value)
        else: # cuando llegamos al que tenga el value
        #se remplaza con algun hijo o tenga o None si no tiene
            if root.left is None: #hijo único derecho
                temp = root.right
                root = None
                return temp

            elif root.right is None: #hijo único izquierdo
                temp = root.left
                root = None
                return temp
            #si tiene dos hijos
            temp = self.leftmost(root.right) 
            # replace root with smallest in right subtree
            root.value = temp.value
            root.right = self.eliminar_nodo(root.right,temp.value)

        #Se actualiza la altura
        root.height = 1 + max(self.get_hight(root.left),
                            self.get_hight(root.right)) 
        # Se calcula el factor de equilibrio 
        factor = self.balanced_factor(root)
 
        # desbalanceado a la derecha
        if factor<-1 and self.balanced_factor(root.right)<=0: 
            return self.rotacion_izquierda(root)
        if factor<-1 and  self.balanced_factor(root.right)>0:
            root.right = self.rotacion_derecha(root.right)
            return self.rotacion_izquierda(root)

        #desbalanceado a la izquierda
        if factor>1 and  self.balanced_factor(root.left)>=0: 
            return self.rotacion_derecha(root)
        if factor>1 and self.balanced_factor(root.left)<0: 
            root.left = self.rotacion_izquierda(root.left)
            return self.rotacion_derecha(root)
        
        return root
        
    # Calculamos el factor de equilibrio 
    def balanced_factor(self, current: User) -> int:
        '''
        Retorna el factor de equilibrio de un nodo

        Args:
            current (Node) : [Node for which we're trying to find the balance factor]

        Returns:
            Node : Returns the balanced factor of the node
        '''
        if not current:
            return 0
        return self.get_hight(current.left) - self.get_hight(current.right)
    
    def get_hight(self, current: User) -> int:
        '''
        Devuelve la altura del arbol/subarbol de donde estemos tomando la raíz

        Args:
            current (Node) : [Node for which we're trying to find the height]
        '''
        if not current:
            return 0
        return current.height
    
    def levelOrderPrint(self, root: User) -> None:
        '''
        Imprime el recorrido por niveles

        Args:
            root (Node) : [Our tree's root]
        '''
        height = self.get_hight(root)

        for level in range(1, height+1):
            print(f'Level {level}', end=': ')
            self.levelOrderTravel(root, level)
            print('\n')

    def levelOrderTravel(self, root: User, level: int) -> None:
        '''
        Recorre el arbol por niveles

        Args:
            root  (Node) : [Node from where we're going to start the moving]
            level (int)  : [Respective level of the node where we started to move]
        '''
        if root is None:
            return
        if level == 1:
            print(f'{root.value}', end=' ')
        elif level > 1:
            self.levelOrderTravel(root.left, level-1)
            self.levelOrderTravel(root.right, level-1)
    
    def rotacion_derecha(self, current: User) -> User:
        '''
        Rotación simple derecha de un arbol balanceado

        Args: 
            current (Node) : [Node where there're unbalanced factor]
        '''
        nueva_raiz = current.left
        current.left = nueva_raiz.right
        nueva_raiz.right = current
        
        # Arreglamos las respectivas alturas de los nodos que estamos moviendo
        current.height = 1 + max(self.get_hight(current.left), self.get_hight(current.right))
        nueva_raiz.height = 1 + max(self.get_hight(nueva_raiz.left), self.get_hight(nueva_raiz.right))
        
        return nueva_raiz
    
    def rotacion_izquierda(self, current: User) -> User:
        '''
        Rotación simple izquierda de un arbol balanceado

        Args:
            current (Node) : [Node where there're unbalanced factor]
        '''
        nueva_raiz = current.right
        current.right = nueva_raiz.left
        nueva_raiz.left = current
        
        # Arreglamos las respectivas alturas de los nodos que estamos moviendo
        current.height = 1 + max(self.get_hight(current.left), self.get_hight(current.right))
        nueva_raiz.height = 1 + max(self.get_hight(nueva_raiz.left), self.get_hight(nueva_raiz.right))
        
        return nueva_raiz
    
    def existe_nodo(self,value)->bool:
        """
        Busca un nodo, avisando si no es hallado
        
        Args:
            value:[value of node to find]

        Returns:
            True [If there is a node with value]
            False [if there is not ]
        """
        node = self.buscar_nodo_recursivo(self.root,value)
        # Usa llamado recursivo para guardar el resultado
        if node is None:
            return False
        else:
            return True
        
    def buscar_nodo(self,value):
        """
        Busca un nodo, avisando si no es hallado
        
        Args:
            value:[value of node to find]
        """
        node = self.buscar_nodo_recursivo(self.root,value)
        # Usa llamado recursivo para guardar el resultado
        if node is None:
            print(f"{value} no fue hallado")

    def buscar_nodo_recursivo(self,root,value):
        """
        Busca recursivamente un nodo
        y muestra la altura e hijos

        Args:
            root(Node): [current node]
            value(int/str): [value of node we search]
        """
        if root is None:
            return None
        else:
            if value<root.value:
                return self.buscar_nodo_recursivo(root.left,value)
            elif value>root.value:
                return self.buscar_nodo_recursivo(root.right,value)
            elif value == root.value:
                print(f"{root} height {root.height} left:{root.left} right {root.right}")
                return root
    
    def leftmost(self,root) -> User:
        """
        Regresa el nodo mas pequeño
        en un (sub)árbol

        Args:
            root(Node): [root of tree]
        """
        P = root
        lvl = 0
        while P.left is not None:
            lvl += 1
            P = P.left
        return P
    
    def buscar_padre(self,value) -> User:
        """
        Busca el padre de nodo con value

        Args:
            value:[value whose father we look for]

        Returns:
            Node:[if there is a father]
            None:[if there cannot be a father]
        """
        if self.root is None:
            print("None")
            return
        if self.root.value == value:
            return None
        else:
            traversed = []
            traversed.append(self.root)
            if self.root.value == value:
                self.root = None
            else:
                while len(traversed) !=0:
                    x = traversed.pop(0)
                    if x.left is not None:
                        if x.left.value == value:
                            return x
                        else:
                            traversed.append(x.left)
                    if x.right is not None:
                        if x.right.value == value:
                            return x
                        else:
                            traversed.append(x.right)

    def uncle(self,value):
        """
        Muestra (si lo tiene) el tío 
        de un nodo

        Args:
            value:[value of node whose uncle we search]
        """
        father = self.buscar_padre(value) #  buscar papá
        if father is None:
            print(f"{value} has no father, therefore no uncle")
        elif father is self.root:
            print("father is root so there is no uncle")
        else:
            traversed = []
            traversed.append(self.root)
            if self.root.value == value: # find grandad select non father-child
                self.root = None
            else:
                while len(traversed) !=0:
                    x = traversed.pop(0)
                    if x.left is not None:
                        if x.left.value == father.value:
                            if x.right is not None:
                                print(f"uncle of {value} is {x.right}")
                                return
                        else:
                            traversed.append(x.left)
                    if x.right is not None:
                        if x.right.value == father.value:
                            if x.left is not None:
                                print(f"uncle of {value} is {x.left}")
                                return
                        else:
                            traversed.append(x.right)

    def grandad(self,value):
        """
        Muestra (si lo tiene) el abuelo de un nodo

        Args:
            value:[value of node whose uncle we search]
        """
        father = self.buscar_padre(value) # determino padre
        if father is None:
            print(f"{value} has no father, therefore no grandad")
        elif father is self.root:
            print("father is root, so there is no grandad")
        else:
            grandad = self.buscar_padre(father.value) # abuelo = padre de padre
            print(f"grandad of {value} is {grandad}")


# Instanciamos el arbol binario balanceado
arbol = ArbolBinarioBalanceado([10,5,15,3,7,13,18,4,2,1])

#arbol = ArbolBinarioBalanceado([3,5,2,4])
#arbol.root = arbol.eliminar_nodo(arbol.root,2)

# Probando el recorrido por niveles
arbol.levelOrderPrint(arbol.root)
arbol.uncle(2)
arbol.grandad(10)
arbol.grandad(7)
arbol.buscar_nodo(143)
'''
Referencias: 
   
    - https://openai.com/blog/chatgpt/
    - https://www.utm.mx/~jahdezp/archivos%20estructuras/Arboles%20AVL.pdf
    - https://es.wikipedia.org/wiki/Rotaci%C3%B3n_de_%C3%A1rboles
    - https://www.geeksforgeeks.org/level-order-tree-traversal/
    
'''

None
2 has no father, therefore no uncle
None
10 has no father, therefore no grandad
None
7 has no father, therefore no grandad
143 no fue hallado


'\nReferencias: \n   \n    - https://openai.com/blog/chatgpt/\n    - https://www.utm.mx/~jahdezp/archivos%20estructuras/Arboles%20AVL.pdf\n    - https://es.wikipedia.org/wiki/Rotaci%C3%B3n_de_%C3%A1rboles\n    - https://www.geeksforgeeks.org/level-order-tree-traversal/\n    \n'

In [88]:
# Nos interesa que no esten duplicados para no tener los nombres repetidos en el diccionario
user_uniques = list(mega_df['User_name'].drop_duplicates())
id_uniques = list(mega_df['User_ID'].drop_duplicates())

# Creamos un diccionario para almacenar los usuarios y su respectiva información
info = {
    'Users': {}
} 

for user, id in zip(user_uniques, id_uniques):
    info['Users'][user] = {'ID': id, 'Valence mean': None}

In [89]:
# Prueba del calculo de la media de manera individual
usuario_prueba = [i for i in mega_df[mega_df['User_ID'] == "usmmuh3bnsqmv3q8d4227ndyt"]['valence']]
media = sum(usuario_prueba)/len(usuario_prueba)

print(f'The mean is {media}')

The mean is 0.5444285714285714


In [90]:
# Prueba del calculo de la media de manera general
for user_id, user_name in zip(mega_df['User_ID'].drop_duplicates(), mega_df['User_name'].drop_duplicates()):
    valence_values = [i for i in mega_df[mega_df['User_ID'] == str(user_id)]['valence']]
    mean = sum(valence_values)/len(valence_values)
    info['Users'][user_name]['Valence mean'] = round(mean, 2)

for user, value in zip(info['Users'], info['Users'].values()):
    print(user, value)

Sebastian Racedo Val {'ID': '12179172375', 'Valence mean': 0.65}
Charlie {'ID': '0cxaozs3a5z8m26vwzzc80wb9', 'Valence mean': 0.53}
Jesús Daniel {'ID': '31wuuvwl5ak5254tirgzgux5hycy', 'Valence mean': 0.35}
carlos andres {'ID': '31oj4ddmhj72gh65gjgkrfbqx4ia', 'Valence mean': 0.73}
jose {'ID': '316aov5an7ol5ldr64mksxnfrwvu', 'Valence mean': 0.72}
Cristhian Agamez {'ID': 'usmmuh3bnsqmv3q8d4227ndyt', 'Valence mean': 0.54}
J {'ID': 'jimenezzuletajhonjanner', 'Valence mean': 0.59}
Juan {'ID': '09n35jjitgmqw8xcqmkp9nsri', 'Valence mean': 0.34}
mcast15 {'ID': 'marianact10-7', 'Valence mean': 0.57}
Maria Isabel Solá Valle {'ID': 'mivallesola', 'Valence mean': 0.46}
K3nnyZY {'ID': '31wmparzjhqbvsn6bcijxbj245cy', 'Valence mean': 0.58}
Jesus Zuluaga Moreno {'ID': 'nm7adjhazf7n85vkf6zxequ7p', 'Valence mean': 0.47}
Laureano Lafaurie Del Villar {'ID': '21nkonqixyujyyuxcgner5p2i', 'Valence mean': 0.51}


In [91]:
nodos = [User(user, info['Users'][user]['ID'], info['Users'][user]['Valence mean']) for user in list(info['Users'].keys())]

for usuario in nodos:
    print(usuario)

[+] User name: Sebastian Racedo Val
    User ID:12179172375
    Valence mean: 0.65
[+] User name: Charlie
    User ID:0cxaozs3a5z8m26vwzzc80wb9
    Valence mean: 0.53
[+] User name: Jesús Daniel
    User ID:31wuuvwl5ak5254tirgzgux5hycy
    Valence mean: 0.35
[+] User name: carlos andres
    User ID:31oj4ddmhj72gh65gjgkrfbqx4ia
    Valence mean: 0.73
[+] User name: jose
    User ID:316aov5an7ol5ldr64mksxnfrwvu
    Valence mean: 0.72
[+] User name: Cristhian Agamez
    User ID:usmmuh3bnsqmv3q8d4227ndyt
    Valence mean: 0.54
[+] User name: J
    User ID:jimenezzuletajhonjanner
    Valence mean: 0.59
[+] User name: Juan
    User ID:09n35jjitgmqw8xcqmkp9nsri
    Valence mean: 0.34
[+] User name: mcast15
    User ID:marianact10-7
    Valence mean: 0.57
[+] User name: Maria Isabel Solá Valle
    User ID:mivallesola
    Valence mean: 0.46
[+] User name: K3nnyZY
    User ID:31wmparzjhqbvsn6bcijxbj245cy
    Valence mean: 0.58
[+] User name: Jesus Zuluaga Moreno
    User ID:nm7adjhazf7n85vkf6zxe

In [97]:
tree = ArbolBinarioBalanceado()

tree.agregar_nodo(nodos[0])
tree.agregar_nodo(nodos[1])
tree.agregar_nodo(nodos[2]) # Se queda sin memoria aca

print(tree.root)

RecursionError: maximum recursion depth exceeded in comparison

In [93]:
# Antes del cambio del User_ID
df

,User_name,User_ID,Artist_name,Track,valence
0,Sebastian Racedo Val,12179172375,Nickelback,Animals,0.818
1,Sebastian Racedo Val,12179172375,Nickelback,Burn It to the Ground,0.600
2,Sebastian Racedo Val,12179172375,Panic! At The Disco,I Write Sins Not Tragedies,0.672
3,Sebastian Racedo Val,12179172375,Fall Out Boy,Thnks fr th Mmrs,0.588
4,Sebastian Racedo Val,12179172375,Hugo,99 Problems,0.592
...,...,...,...,...,...
95,J,jimenezzuletajhonjanner,El Cuarteto De Nos,Ya No Sé Que Hacer Conmigo,0.502
96,J,jimenezzuletajhonjanner,El Cuarteto De Nos,Mario Neta,0.418
97,J,jimenezzuletajhonjanner,El Cuarteto De Nos,Punta Cana,0.948
98,J,jimenezzuletajhonjanner,Residente,Pecador,0.465


In [94]:
def new_id(text: str) -> str:
    '''
    Devuelve un ID en su formato ASCII

    Args:
        text (str) : [Text that we'll transform into ASCII representation]
    '''
    carSplit = [letra for letra in text]
    ascii_rep = [ord(k) for k in carSplit]
    nuevoID = ''.join([str(i) for i  in ascii_rep])

    return nuevoID

In [95]:
for k in range(len(df['User_ID'])):
    df['User_ID'][k] = new_id(df['User_ID'][k])

# Despues del cambio del User_ID
df
    

/tmp/ipykernel_5426/1548084461.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['User_ID'][k] = new_id(df['User_ID'][k])


,User_name,User_ID,Artist_name,Track,valence
0,Sebastian Racedo Val,4950495557495550515553,Nickelback,Animals,0.818
1,Sebastian Racedo Val,4950495557495550515553,Nickelback,Burn It to the Ground,0.600
2,Sebastian Racedo Val,4950495557495550515553,Panic! At The Disco,I Write Sins Not Tragedies,0.672
3,Sebastian Racedo Val,4950495557495550515553,Fall Out Boy,Thnks fr th Mmrs,0.588
4,Sebastian Racedo Val,4950495557495550515553,Hugo,99 Problems,0.592
...,...,...,...,...,...
95,J,1061051091011101011221221171081011169710610411...,El Cuarteto De Nos,Ya No Sé Que Hacer Conmigo,0.502
96,J,1061051091011101011221221171081011169710610411...,El Cuarteto De Nos,Mario Neta,0.418
97,J,1061051091011101011221221171081011169710610411...,El Cuarteto De Nos,Punta Cana,0.948
98,J,1061051091011101011221221171081011169710610411...,Residente,Pecador,0.465
